<a href="https://colab.research.google.com/github/Sebastian310/Rpa.pyautogui/blob/main/Analisis_de_modelos_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd; from xgboost import XGBClassifier; import numpy as np; import xgboost as xgb; from sklearn.metrics import roc_auc_score; from sklearn.linear_model import LogisticRegression
import warnings;warnings.filterwarnings("ignore") # Se ignora advertencias de sklearn

In [ ]:
# Carga de datos y eliminación de filas con valores "NA"
heart = pd.read_csv('Heart.csv')
Dataset_SIN_NA = heart.dropna()

print("Cantidad para muestra 80% es :", round(len(Dataset_SIN_NA) * 0.8)," y Cantidad para Testeo 20% es :",round(len(Dataset_SIN_NA) * 0.2) )

# Creación de variables dummies para variables Cualitativas y reemplazo de valores en variable objetivo
randomised_heart = pd.get_dummies(Dataset_SIN_NA, columns=['ChestPain', 'Thal'], drop_first=False)
randomised_heart['AHD'] = randomised_heart['AHD'].replace({'Yes': 1, 'No': 0})

# distibuto 80% de datos entrenamiento y 20% de prueba
heart_Train2, heart_Test2 = randomised_heart.loc[0:238], randomised_heart.loc[238:]

# Asigno variables a Dataset de prueba
columns_x_Var = ['Age', 'Sex', 'RestBP', 'Chol', 'Fbs', 'RestECG', 'MaxHR', 'ExAng', 'Oldpeak', 'Slope', 'ChestPain_asymptomatic', 'ChestPain_nonanginal', 'ChestPain_nontypical', 'ChestPain_typical','Thal_fixed', 'Thal_normal']
x_entreno, x_prueba = heart_Train2[columns_x_Var], heart_Test2[columns_x_Var]
y_entreno, y_prueba = heart_Train2['AHD'], heart_Test2['AHD']


Cantidad para muestra 80% es : 238  y Cantidad para Testeo 20% es : 59


Ejercicio 1- Haz validación cruzada 80%-20% utilizando regresión logística (sin penalización

1.   Evalúa el test utilizando la AUC
2.   Utiliza variables de input numéricas y categoricas (transformadas usando la funcion get_dummies
3.   Hacer la validación cruzada aleatoriamente con 5-fold







In [ ]:
clf = LogisticRegression(penalty='none')    # Creación del modelo de Regresión Logística

X, y = pd.concat([x_entreno, x_prueba]), pd.concat([y_entreno, y_prueba]) # Variables de entrenamiento y prueba

k_divisiones = 5  # Número de divisiones para la validación cruzada

fold_size = len(X) // k_divisiones  # Tamaño de cada división

auc_scores = []  # Lista para almacenar los puntajes AUC

for i in range(k_divisiones):  # División de los datos en entrenamiento y prueba
    X_test = X.iloc[i*fold_size:(i+1)*fold_size]
    y_test = y.iloc[i*fold_size:(i+1)*fold_size]
    X_train = X.drop(X_test.index)
    y_train = y.drop(y_test.index)

    clf.fit(X_train, y_train)  # Entrenamiento del modelo

    y_prob = clf.predict_proba(X_test)[:, 1]                             # Calcula las probabilidades
    auc_scores.append(roc_auc_score(y_test, y_prob))

print('Promedio AUC: ', np.mean(auc_scores))


Promedio AUC:  0.893346174698167


Ejersicio 2 - Haz validación cruzada 80%-20% utilizando el modelo XGBoost  Con puntos  

1.   Prueba diferentes combinaciones de los hyperparámetros max_depth y n_estimators.

1.   Utiliza bucles for para evaluar la validación cruzada en cada uno de los hyperparámetros.
2.   Utiliza variables de input numéricas y categoricas (transformadas usando la funcion get_dummies

2.   Evalúa el test utilizando la AUC


In [ ]:
results = []
X = randomised_heart[['Age', 'Sex', 'RestBP', 'Chol', 'Fbs', 'RestECG', 'MaxHR', 'ExAng', 'Oldpeak', 'Slope','ChestPain_asymptomatic', 'ChestPain_nonanginal', 'ChestPain_nontypical', 'ChestPain_typical','Thal_fixed', 'Thal_normal']]
y = randomised_heart['AHD'].to_frame()            # Convierto esta serie a Dataframe para usar posteriormente Iloc

tamano_fold = len(X) // 5   # Definir el tamaño de los pliegues

for N_estimator in [10, 50, 100,150,200,]:
    for depth in range(1, 8):
        clf = XGBClassifier(n_estimators=N_estimator, learning_rate=1.0, max_depth=depth, random_state=0)

        clf_scores = [] ; auc_scores = []  # designo listas

        for i in range(5):

            # Crear los índices de entrenamiento y prueba
            test_indices = list(range(i * tamano_fold, (i + 1) * tamano_fold))
            train_indices = list(set(range(len(X))) - set(test_indices))

            # Crear los conjuntos de entrenamiento y prueba
            X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
            y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

            # Entrenamiento del modelo
            clf.fit(X_train, y_train)

            # Calcula el score del clasificador
            clf_score = clf.score(X_test, y_test)
            clf_scores.append(clf_score)

            # Calcula el AUC
            y_prob = clf.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_prob)
            auc_scores.append(auc)

        # Guardar los resultados en la lista "Results"
        results.append({'N_estimator': N_estimator,'depth': depth,'clf_score': np.mean(clf_scores),'AUC': np.mean(auc_scores)})

# Imprimir los resultados
for result in results:
    print('N_estimator:', result['N_estimator'], 'depth:', result['depth'], 'clf_score:', result['clf_score'], 'AUC:', result['AUC'])


N_estimator: 10 depth: 1 clf_score: 0.7898305084745763 AUC: 0.8852227799246053
N_estimator: 10 depth: 2 clf_score: 0.7457627118644068 AUC: 0.8432825845814296
N_estimator: 10 depth: 3 clf_score: 0.7593220338983051 AUC: 0.8290629480276147
N_estimator: 10 depth: 4 clf_score: 0.7559322033898305 AUC: 0.831890096891193
N_estimator: 10 depth: 5 clf_score: 0.7491525423728813 AUC: 0.8529189626499383
N_estimator: 10 depth: 6 clf_score: 0.7389830508474577 AUC: 0.8410421052111003
N_estimator: 10 depth: 7 clf_score: 0.7389830508474576 AUC: 0.8268085073271392
N_estimator: 50 depth: 1 clf_score: 0.7898305084745763 AUC: 0.8873880054013371
N_estimator: 50 depth: 2 clf_score: 0.7457627118644068 AUC: 0.8251746985012052
N_estimator: 50 depth: 3 clf_score: 0.7525423728813558 AUC: 0.8362561490863362
N_estimator: 50 depth: 4 clf_score: 0.742372881355932 AUC: 0.8268025842241638
N_estimator: 50 depth: 5 clf_score: 0.7559322033898305 AUC: 0.8481822022705521
N_estimator: 50 depth: 6 clf_score: 0.7559322033898306